In [2]:
import dill
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

In [3]:
results_dict = {}
for i in range(5):
    with open(f"xz_results_{i}.pkl", "rb") as dill_file:
        results_dict[f"xz_{i}"] = dill.load(dill_file)
    with open(f"sch_results_{i}.pkl", "rb") as dill_file:
        results_dict[f"sch_{i}"] = dill.load(dill_file)

In [19]:

# * Empty node degree centrality analysis (do empty nodes congregate in high-degree areas?)

dc_ratios = {}
for i in range(5):
    results = results_dict[f"xz_{i}"]
    dc_ratio = []
    for k in range(100):
        G = results[k].graph
        dc = nx.degree_centrality(G)
        dc_empty_nodes = []
        for node in G.nodes:
            if G.nodes[node]['object'] == None:
                dc_empty_nodes.append(dc[node])
        dc_ratio.append(np.mean(dc_empty_nodes))
    dc_ratios[f"Xie & Zhou, {i*32}"]= dc_ratio
    
    results = results_dict[f"sch_{i}"]
    dc_ratio = []
    for k in range(100):
        G = results[k].graph
        dc = nx.degree_centrality(G)
        dc_empty_nodes = []
        for node in G.nodes:
            if G.nodes[node]['object'] == None:
                dc_empty_nodes.append(dc[node])
        dc_ratio.append(np.mean(dc_empty_nodes))
    dc_ratios[f"Schelling, {i*32}"] = dc_ratio


fig, ax = plt.subplots()
ax.boxplot(dc_ratios.values(),)
ax.set_xticklabels(dc_ratios.keys())
ax.set_ylabel('Vacancy Centrality Coeffecient Ratio')
ax.set_xlabel('Method, Densifications')
ax.set_title('Impact of Densifications on Vacancy Degree Centrality')
plt.show()

In [20]:

# * Node degree centrality analysis (do tolerant nodes congregate in high-degree areas?)

dc_ratios = {}
for i in range(5):
    results = results_dict[f"xz_{i}"]
    dc_tolerant = []
    dc_intolerant = []
    for k in range(100):
        G = results[k].graph
        dc = nx.degree_centrality(G)
        dc_tolerant_nodes = []
        dc_intolerant_nodes = []
        for node in G.nodes:
            if G.nodes[node]['object'] != None and G.nodes[node]['object'].tolerance_threshold != None:
                if G.nodes[node]['object'].tolerance_threshold >= 0.57:
                    dc_tolerant_nodes.append(dc[node])
                elif G.nodes[node]['object'].tolerance_threshold <= 0.25:
                    dc_intolerant_nodes.append(dc[node])
        dc_tolerant.append(np.mean(dc_tolerant_nodes))
        dc_intolerant.append(np.mean(dc_intolerant_nodes))
    dc_ratios[f"Tolerant, {i*32}"]= dc_tolerant
    dc_ratios[f"Intolerant, {i*32}"]= dc_intolerant


fig, ax = plt.subplots()
ax.boxplot(dc_ratios.values(),)
ax.set_xticklabels(dc_ratios.keys())
ax.set_ylabel('Degree Centrality Coeffecient')
ax.set_xlabel('Type, Densifications')
ax.set_title('Impact of Densifications on Degree Centrality, Tolerant vs. Intolerant Individuals')
plt.show()

In [104]:

# * Plots change in assortativity (Xie & Zhou)


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

color = iter(plt.cm.Oranges(np.linspace(.7, 1, 5)))
for i in range(5):
    results = results_dict[f"sch_{i}"]
    array = np.array([(r.attribute_assortativity) for r in results])
    average_result = np.average(array,axis=0)
    c = next(color)
    ax.plot(range(4001),average_result, color=c, zs=i*32, zdir='y')


color = iter(plt.cm.Blues(np.linspace(.7, 1, 5)))
for i in range(5):
    results = results_dict[f"xz_{i}"]
    array = np.array([(r.attribute_assortativity) for r in results])
    average_result = np.average(array,axis=0)
    c = next(color)
    ax.plot(range(4001),average_result, color=c, zs=i*32, zdir='y')


ax.set_title('Change in Assortativity')
ax.set_xlabel('     Time')
ax.set_xticks([0,1000,2000,3000,4000])
ax.set_ylabel('Densifications')
ax.set_yticks(np.arange(0,129,32))
ax.set_zlabel('Assortativity')
plt.show()

In [37]:

# * Plots n Final Connected Components (Xie & Zhou)

fig, ax  = plt.subplots()

plt_dict = {}
for i in range(5):
    results = results_dict[f"xz_{i}"]
    array = np.array([(r.n_connected_components[-1]) for r in results])
    plt_dict[f'Xie & Zhou, {i*32}'] = list(array)
    results = results_dict[f"sch_{i}"]
    array = np.array([(r.n_connected_components[-1]) for r in results])
    plt_dict[f'Schelling, {i*32}'] = list(array)

ax.boxplot(plt_dict.values(),)
ax.set_xticklabels(plt_dict.keys())
ax.set_title('Final n Connected Components')
ax.set_xlabel('Method, Densifications')
ax.set_ylabel('n Connected Components')
plt.show()

In [38]:

# * Plots Final CC Size

fig, ax  = plt.subplots()

plt_dict = {}
for i in range(5):
    results = results_dict[f"xz_{i}"]
    array = np.array([(r.avg_cc_size[-1]) for r in results])
    plt_dict[f'Xie & Zhou, {i*32}'] = list(array)
    results = results_dict[f"sch_{i}"]
    array = np.array([(r.avg_cc_size[-1]) for r in results])
    plt_dict[f'Schelling, {i*32}'] = list(array)

ax.boxplot(plt_dict.values(),)
ax.set_xticklabels(plt_dict.keys())
ax.set_title('Final Mean Connected Component Size')
ax.set_xlabel('Method, Densifications')
ax.set_ylabel('n Connected Components')
plt.show()

In [36]:

# * Plots degree distributions by n densifications

color = iter(plt.cm.rainbow(np.linspace(0, 1, 5)))
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

for i in range(5):
    results = results_dict[f"sch_{i}"]
    c = next(color)
    for k in range(100):
        dh = nx.degree_histogram(results[k].graph)
        plt.plot(list(range(len(dh))),dh, c=c, zs=i*32, zdir='y')
ax.set_title('Impact of Densifications on Degree Distribution')
ax.set_xlabel('Degree')
ax.set_ylabel('Densifications')
ax.set_zlabel('n nodes')
ax.set_yticks(np.arange(0,129,32))
plt.show()

In [51]:

# * Plots mean degree observed by n densifications.

color = iter(plt.cm.rainbow(np.linspace(0, 1, 5)))
fig = plt.figure()
mean_list = []
min_list = []
max_list = []
for i in range(5):
    results = results_dict[f"xz_{i}"]
    c = next(color)
    average = []
    minim = []
    maxim = []
    for k in range(100):
        average.append(np.mean([x[1] for x in list(results[k].graph.degree)]))
        minim.append(min([x[1] for x in list(results[k].graph.degree)]))
        maxim.append(max([x[1] for x in list(results[k].graph.degree)]))
    mean_list.append(np.mean(average))
    min_list.append(np.mean(minim))
    max_list.append(np.mean(maxim))
    plt.scatter(i*32,np.mean(average))
plt.title("Impact of Densifications on Mean Degree")
plt.ylabel("Mean Degree")
plt.xlabel("Densifications")
plt.xticks([n*32 for n in np.arange(0,5)])
print(mean_list)
print(min_list)
print(max_list)

[3.875, 7.1498046875, 10.02955078125, 12.4845703125, 14.72921875]
[2.0, 2.0, 2.0, 2.02, 2.16]
[4.0, 28.25, 32.3, 34.91, 36.37]


In [6]:

# * Draws selected network displays individual color.

plt.figure()
ax = plt.gca()
#ax.set_title("Xie & Zhou, 0 Densifications")
G = results_dict["sch_2"][0].graph
color_map = []
pos = dict((n,n) for n in G.nodes)
for node in G.nodes:
    if G.nodes[node]['object'] == None:
        color_map.append('grey')
    elif G.nodes[node]['object'].color == 0:
        color_map.append('red')
    else:
        color_map.append('blue')
nx.draw(G, pos = pos, node_size=30, node_color= color_map, with_labels=False, ax=ax)
plt.show()

In [4]:
# * Draws selected network.

plt.figure()
ax = plt.gca()
#ax.set_title("Xie & Zhou, 0 Densifications")
G = results_dict["sch_2"][0].graph
color_map = []
pos = dict((n,n) for n in G.nodes)
for node in G.nodes:
    if G.nodes[node]['object'] == None:
        color_map.append('black')
    elif G.nodes[node]['object'].tolerance_threshold != None:
        color_map.append(plt.cm.RdYlGn(G.nodes[node]['object'].tolerance_threshold))
    else:
        color_map.append('yellow')

nx.draw(G, pos = pos, node_size=30, node_color= color_map, with_labels=False, ax=ax)
plt.show()

C:\Users\deter\AppData\Roaming\Python\Python39\site-packages\matplotlib\cbook\__init__.py:1096: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.asanyarray(x)


In [14]:

# * Results table

col_heads_xz = ['Densifications', 'Mean Assortativity', 'Std Assortativity', 'Mean Connected Components', 'Std Connected Components', 'Mean Vacancy Degree Centrality', 'Std Vacancy Degree Centrality','Mean T Node Degree Centrality', 'Std T Node Degree Centrality','Mean I Node Degree Centrality','Std I Node Degree Centrality']
col_heads_sch = ['Densifications', 'Mean Assortativity', 'Std Assortativity', 'Mean Connected Components', 'Std Connected Components', 'Mean Vacancy Degree Centrality', 'Std Vacancy Degree Centrality']
xz_df = pd.DataFrame(columns=col_heads_xz)
sch_df = pd.DataFrame(columns=col_heads_sch)
for i in range(5):
    results = results_dict[f"xz_{i}"]
    densifications = int(i * 32)
    mean_assortativity = np.mean([(r.attribute_assortativity[-1]) for r in results])
    std_assortativity = np.std([(r.attribute_assortativity[-1]) for r in results])
    mean_cc = np.mean([(r.n_connected_components[-1]) for r in results])
    std_cc = np.std([(r.n_connected_components[-1]) for r in results])
    mean_graph_vacancy_dc = []
    for k in range(100):
        G = results[k].graph
        dc = nx.degree_centrality(G)
        dc_empty_nodes = []
        for node in G.nodes:
            if G.nodes[node]['object'] == None:
                dc_empty_nodes.append(dc[node])
        mean_graph_vacancy_dc.append(np.mean(dc_empty_nodes))
    mean_population_vacancy_dc = np.mean(mean_graph_vacancy_dc)
    std_population_vacancy_dc = np.std(mean_graph_vacancy_dc)

    mean_graph_T_dc = []
    mean_graph_I_dc = []
    for k in range(100):
        G = results[k].graph
        dc = nx.degree_centrality(G)
        dc_tolerant_nodes = []
        dc_intolerant_nodes = []
        for node in G.nodes:
            if G.nodes[node]['object'] != None and G.nodes[node]['object'].tolerance_threshold != None:
                if G.nodes[node]['object'].tolerance_threshold >= 0.57:
                    dc_tolerant_nodes.append(dc[node])
                elif G.nodes[node]['object'].tolerance_threshold <= 0.07:
                    dc_intolerant_nodes.append(dc[node])
        mean_graph_T_dc.append(np.mean(dc_tolerant_nodes))
        mean_graph_I_dc.append(np.mean(dc_intolerant_nodes))
    mean_population_T_dc = np.mean(mean_graph_T_dc)
    std_population_T_dc = np.std(mean_graph_T_dc)
    mean_population_I_dc = np.mean(mean_graph_I_dc)
    std_population_I_dc = np.std(mean_graph_I_dc)

    xz_df.loc[len(xz_df.index)] = [densifications, mean_assortativity, std_assortativity, mean_cc, std_cc, mean_population_vacancy_dc, std_population_vacancy_dc, mean_population_T_dc, std_population_T_dc, mean_population_I_dc, std_population_I_dc]


for i in range(5):
    results = results_dict[f"sch_{i}"]
    densifications = int(i * 32)
    mean_assortativity = np.mean([(r.attribute_assortativity[-1]) for r in results])
    std_assortativity = np.std([(r.attribute_assortativity[-1]) for r in results])
    mean_cc = np.mean([(r.n_connected_components[-1]) for r in results])
    std_cc = np.std([(r.n_connected_components[-1]) for r in results])
    mean_graph_vacancy_dc = []
    for k in range(100):
        G = results[k].graph
        dc = nx.degree_centrality(G)
        dc_empty_nodes = []
        for node in G.nodes:
            if G.nodes[node]['object'] == None:
                dc_empty_nodes.append(dc[node])
        mean_graph_vacancy_dc.append(np.mean(dc_empty_nodes))
    mean_population_vacancy_dc = np.mean(mean_graph_vacancy_dc)
    std_population_vacancy_dc = np.std(mean_graph_vacancy_dc)


    sch_df.loc[len(sch_df.index)] = [densifications, mean_assortativity, std_assortativity, mean_cc, std_cc, mean_population_vacancy_dc, std_population_vacancy_dc]

In [21]:
xz_df.to_excel('xz_results.xlsx')

In [23]:
sch_df.to_excel('sch_results.xlsx')